In [1]:
from keras.layers import Dense, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from keras import models
from keras.preprocessing.image import ImageDataGenerator
from keras.metrics import categorical_crossentropy

Using TensorFlow backend.


In [2]:
my_model= models.Sequential()

my_model.add(Conv2D(32, (3, 3), activation='relu', padding='same', 
                    input_shape=(224,224,3)))
my_model.add(MaxPooling2D((2, 2), padding='same'))

my_model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
my_model.add(MaxPooling2D((2, 2), padding='same'))

my_model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
my_model.add(MaxPooling2D((2, 2), padding='same'))

my_model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
my_model.add(MaxPooling2D((2, 2), padding='same'))


my_model.add(GlobalAveragePooling2D())

my_model.add(Dense(256, activation='relu'))
my_model.add(BatchNormalization())

my_model.add(Dense(25, activation='softmax'))

In [5]:
my_model.compile(optimizer='adam', loss='categorical_crossentropy', 
                 metrics=['accuracy'])

In [0]:
import requests

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)
    print(token)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk:
                f.write(chunk)


In [0]:
data_generator = ImageDataGenerator(rescale=1./255)

In [31]:
train_generator = data_generator.flow_from_directory(
        '/content/images_train',
        target_size=(224, 224),
        batch_size=64,
        class_mode='categorical',
        classes=[str(i) for i in range(25)]
        )

validation_generator = data_generator.flow_from_directory(
        '/content/images_val',
        target_size=(224, 224),
        batch_size=64,
        class_mode='categorical',
        classes=[str(i) for i in range(25)]
        )



Found 3000 images belonging to 25 classes.
Found 750 images belonging to 25 classes.


In [0]:
train_generator.class_indices

In [0]:

history = my_model.fit_generator(
        train_generator,
        epochs=100,
        validation_data=validation_generator)

In [0]:
my_model.save('model.h5')

In [51]:
my_model.evaluate_generator(validation_generator)

[0.24400990454355875, 0.946666667620341]